# Google Drive Tools

In [ ]:
from autogen import AssistantAgent, LLMConfig
from autogen.tools import tool
from autogen.tools.experimental.google import GoogleCredentialsLocalProvider, GoogleDriveToolMap

In [ ]:
client_secret_file = "../client_secret_ag2.json"

# This is the token file that will be created after the first run
# of the script. It will be used to authenticate the user in the future.
token_file = "../my_token.json"

provider = GoogleCredentialsLocalProvider(
    client_secret_file=client_secret_file,
    scopes=GoogleDriveToolMap.recommended_scopes(),
    token_file=token_file,
)

credentials = provider.get_credentials()

In [ ]:
llm_config = LLMConfig.from_json(
    path="OAI_CONFIG_LIST",
).where(model="gpt-4o-mini")

assistant = AssistantAgent(name="assistant", llm_config=llm_config)

In [ ]:
tool_map = GoogleDriveToolMap(
    credentials=credentials,
    download_folder="ag2_drive_downloads",
)

tool_map.register_for_llm(assistant)

In [ ]:
assistant.run(
    message="""Get me the last 5 files and download all google-apps.document meme types.
Ignore subfolders for now.
Once done, write 'TERMINATE'.""",
    max_turns=5,
    tools=tool_map.tools,
    user_input=False,
)

## Create Custom GoogleDriveToolMap

In [ ]:
class MyGoogleDriveToolMap(GoogleDriveToolMap):
    def __init__(
        self,
        *,
        credentials,
        download_folder,
    ):
        super().__init__(credentials=credentials, download_folder=download_folder)

        @tool(description="Create summary of completed downloads")
        def create_summary(summary: str):
            # Your implementation here
            return "Summary created"

        self.set_tool(create_summary)

In [ ]:
assistant = AssistantAgent(name="assistant", llm_config=llm_config)

tool_map = MyGoogleDriveToolMap(
    credentials=credentials,
    download_folder="ag2_drive_downloads",
)

tool_map.register_for_llm(assistant)

assistant.run(
    message="""Get me the last 5 files and download all google-apps.document meme types.
Ignore subfolders for now.
Once done, create a summary of the downloads.""",
    max_turns=5,
    tools=tool_map.tools,
    user_input=False,
)